<a href="https://colab.research.google.com/github/DeepManuPy/convNets/blob/master/StandardModels/ImplementingModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.models import Model
from keras.layers import Conv2D,MaxPool2D,Input,Activation
from keras.utils import plot_model
from keras.layers.merge import concatenate,add

In [2]:
def vgg_block(layer_in,n_filters,n_conv):
  for _ in range(n_conv):
    layer_in = Conv2D(n_filters,(3,3),padding="same",activation="relu")(layer_in)
  
  layer_in = MaxPool2D((2,2),strides=(2,2))(layer_in)
  return layer_in

visible = Input(shape=(256,256,3))
layer = vgg_block(visible,64,2)
layer = vgg_block(layer,128,2)
layer = vgg_block(layer,256,4)

# Create the Model
model = Model(inputs=visible,outputs=layer)

model.summary()
plot_model(model,show_shapes=True,to_file="Multi-VGG-Blocks.png")

W0622 17:35:45.622247 140525074704256 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0622 17:35:45.669601 140525074704256 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0622 17:35:45.677539 140525074704256 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0622 17:35:45.718695 140525074704256 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 64, 128)       0         
__________

In [8]:
def naive_inception_module(layer_in,f1,f2,f3):
  # 1x1 layer
  conv1 = Conv2D(f1,(1,1),padding="same",activation="relu")(layer_in)
  # 3x3 Conv
  conv3 = Conv2D(f2,(3,3),padding="same",activation="relu")(layer_in)
  # 5x5 Conv
  conv5 = Conv2D(f3,(5,5),padding="same",activation="relu")(layer_in)
  # Concatenate the filters
  layer_out = concatenate([conv1,conv3,conv5],axis=-1)
  layer_out = MaxPool2D((2,2),strides=(2,2))(layer_out)
  return layer_out

visible = Input(shape=(256,256,3))
layer_out = naive_inception_module(visible,128,128,128)
model = Model(inputs=visible,outputs=layer_out)

model.summary()
plot_model(model,show_shapes=True,to_file="Naive-Inception_module.png")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 256, 256, 128 512         input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 256, 256, 128 3584        input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 256, 256, 128 9728        input_5[0][0]                    
__________________________________________________________________________________________________
concatenat

In [7]:
def residual_module(layer_in,n_filters):
  merge_input = layer_in
  if layer_in.shape[-1] != n_filters:
    merge_input = Conv2D(n_filters,(1,1),padding="same",activation="relu")(layer_in)
  conv1 = Conv2D(n_filters,(3,3),padding="same",activation="relu")(layer_in)
  conv2 = Conv2D(n_filters,(3,3),padding="same",activation="linear")(conv1)
  layer_out = add([conv2,merge_input])
  layer_out = Activation("relu")(layer_out)
  return layer_out

visible = Input(shape=(256,256,3))
layer = residual_module(visible,64)
model = Model(inputs=visible,outputs=layer)

model.summary()
plot_model(model,show_shapes=True,to_file="Residual Module.png")


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 256, 256, 64) 1792        input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_17 (Conv2D)              (None, 256, 256, 64) 36928       conv2d_16[0][0]                  
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 256, 256, 64) 256         input_4[0][0]                    
__________________________________________________________________________________________________
add_1 (Add